# <div style="text-align:center;background-color:#f1f4f7;"><hr>SENSOCEAN - DATA ANALYZER<hr></div>


In [1]:
import sys
sys.path

import sys
!{sys.executable} -m pip install numpy pandas matplotlib folium datetime


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: C:\Users\kamel\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


### Librairies

In [2]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import patches
import folium
import folium.plugins
import datetime

# 1 - Préparation des fichiers
-----------------------------------------------------------------------------
----------


## Lecture /assemblage du/des fichiers CSV

On récupére le fichier CSV sur la carte SD, et on l'ouvre ici avec pandas. On affiche l'entête du fichier et les premières lignes.

In [17]:
data = pd.read_csv(r'C:\Users\kamel\Desktop\IMT Atlantique\A2\Commande Entreprise\combine3.csv', delimiter=';')

In [18]:
data.head()

,Lat,Lng,Date,Time,Bat %,Bat mV,Pression_ext(hpa),Temp_ext(C),Temp_int(C),Temp_sea(C),EC_sea,Temp_sea(C) .1,EC_sea .1,Temp_sea(C) .2,EC_sea .2
0,64.151687,-21.932544,21/08/03,09:15:14,100,4097,1005.509,12.45,12.250,-1023,39830,13.51,39840,13.52,39870
1,64.151752,-21.932455,21/08/03,09:25:49,100,4095,1005.621,12.60,12.438,13.528,39850,13.541,39850,13.547,39880
2,64.151746,-21.932064,21/08/03,09:35:59,100,4097,1005.574,12.72,12.500,13.553,39900,13.558,39920,13.561,39950
3,64.151808,-21.932357,21/08/03,09:46:08,100,4104,1005.609,12.73,12.562,13.573,39960,13.581,39980,13.589,40020
4,64.152692,-21.930707,21/08/03,09:56:17,100,4100,1005.599,-136.72,12.688,13.588,39890,13.598,39860,13.582,39920


In [19]:
print("Nombre de ligne : ", len(data))

Nombre de ligne :  5365


## Filtrage des données

### Dates/heure : Identification et suppression des dates érronnées

In [20]:
# définition des fonction d'identification des mauvais format de  dates
error_index=[]

def val_date(date_text):
    try:
        datetime.datetime.strptime(date_text, ' %y/%m/%d ')
    except :
        print("Incorrect data format, should be YY/MM/DD : row =", i) 
        if i not in error_index :
            error_index.append(i)

def val_time(time_text):
    try:
        datetime.datetime.strptime(time_text, ' %H:%M:%S ')
    except :
        print("Incorrect data format, should be HH:MM:SS : row =", i)
        if i not in error_index :
            error_index.append(i)

In [21]:
# identification des erreurs et création d'une liste d'index de lignes erronés
for i in range(len(data)):
    val_date(data.loc[i][' Date '])
    val_time(data.loc[i][' Time '])
    
error_index.sort()
print('Error Index : ', error_index)
print('Nombre de date erroné =',len(error_index))
print('Pourcentage de perte = ', len(error_index)*100/len(data),"%")




Incorrect data format, should be HH:MM:SS : row = 274
Incorrect data format, should be HH:MM:SS : row = 462
Incorrect data format, should be HH:MM:SS : row = 474
Incorrect data format, should be YY/MM/DD : row = 685
Incorrect data format, should be YY/MM/DD : row = 690
Incorrect data format, should be HH:MM:SS : row = 720
Incorrect data format, should be YY/MM/DD : row = 799
Incorrect data format, should be YY/MM/DD : row = 801
Incorrect data format, should be HH:MM:SS : row = 1076
Incorrect data format, should be HH:MM:SS : row = 1561
Incorrect data format, should be HH:MM:SS : row = 1570
Incorrect data format, should be HH:MM:SS : row = 1792
Incorrect data format, should be HH:MM:SS : row = 1805
Incorrect data format, should be HH:MM:SS : row = 1910
Incorrect data format, should be HH:MM:SS : row = 1994
Incorrect data format, should be HH:MM:SS : row = 2002
Incorrect data format, should be HH:MM:SS : row = 2161
Incorrect data format, should be HH:MM:SS : row = 2182
Incorrect data for

In [22]:
# suppression des lignes erronés dans le data frame
for i in range(len(data)):
    if i in error_index :
        data = data.drop(i)
        
print("Nombre de ligne restante : ", len(data))

Nombre de ligne restante :  5309


Pour les mesures T & S de la mer. Les valeurs qui ne respectent pas les conditions suivantes seront supprimées :
- Absence de valeurs
- Valeur nulles
- Autre valeur non numériques
- Valeurs de température = -1023 
- Température non comprise dans l'intervalle [-20; 100]°C 
- Salinité non comprise dans l'intervalle [10; 80]
Le filtrage doit s'opérer sur  toutes les colonnes de température et salinité. Le filtrage consitera à supprimer la lignes correspondantes

Filtrage des valeurs de température interne ou externe ou pression :
- Les valeurs non numérique en température et en pression
- Les valeurs de température en dehors de l'intervalle [-150;+150]°C
- les valeurs de pression en dehors de l'intervalle [700; 2000] hpa

### Suppression des lignes ne présentant aucune valeur

In [23]:
data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5309 entries, 0 to 5364
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Lat                  5309 non-null   float64
 1    Lng                 5309 non-null   float64
 2    Date                5309 non-null   object 
 3    Time                5309 non-null   object 
 4    Bat %               5309 non-null   int64  
 5    Bat mV              5309 non-null   int64  
 6    Pression_ext(hpa)   5309 non-null   float64
 7    Temp_ext(C)         5309 non-null   float64
 8    Temp_int(C)         5309 non-null   float64
 9    Temp_sea(C)         5309 non-null   object 
 10   EC_sea              5309 non-null   object 
 11   Temp_sea(C) .1      5309 non-null   object 
 12   EC_sea .1           5309 non-null   object 
 13   Temp_sea(C) .2      5309 non-null   object 
 14   EC_sea .2           5309 non-null   object 
dtypes: float64(5), int64(2), object(8)
memory u

### Suppression des duplications

In [24]:
data.drop_duplicates(inplace = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5309 entries, 0 to 5364
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Lat                  5309 non-null   float64
 1    Lng                 5309 non-null   float64
 2    Date                5309 non-null   object 
 3    Time                5309 non-null   object 
 4    Bat %               5309 non-null   int64  
 5    Bat mV              5309 non-null   int64  
 6    Pression_ext(hpa)   5309 non-null   float64
 7    Temp_ext(C)         5309 non-null   float64
 8    Temp_int(C)         5309 non-null   float64
 9    Temp_sea(C)         5309 non-null   object 
 10   EC_sea              5309 non-null   object 
 11   Temp_sea(C) .1      5309 non-null   object 
 12   EC_sea .1           5309 non-null   object 
 13   Temp_sea(C) .2      5309 non-null   object 
 14   EC_sea .2           5309 non-null   object 
dtypes: float64(5), int64(2), object(8)
memory u

Changement des types des colonnes 'str' en 'float' et suppression des lignes avec des chaines de caractères vides

In [25]:
data[' Temp_sea(C) '] = pd.to_numeric(data[' Temp_sea(C) '],errors='coerce')
data[' Temp_sea(C) .1'] = pd.to_numeric(data[' Temp_sea(C) .1'],errors='coerce')
data[' Temp_sea(C) .2'] = pd.to_numeric(data[' Temp_sea(C) .2'],errors='coerce')
data[' EC_sea '] = pd.to_numeric(data[' EC_sea '],errors='coerce')
data[' EC_sea .1'] = pd.to_numeric(data[' EC_sea .1'],errors='coerce')
data[' EC_sea .2'] = pd.to_numeric(data[' EC_sea .2'],errors='coerce')

In [26]:
data.reset_index(inplace=True, drop=True)

data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5304 entries, 0 to 5308
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Lat                  5304 non-null   float64
 1    Lng                 5304 non-null   float64
 2    Date                5304 non-null   object 
 3    Time                5304 non-null   object 
 4    Bat %               5304 non-null   int64  
 5    Bat mV              5304 non-null   int64  
 6    Pression_ext(hpa)   5304 non-null   float64
 7    Temp_ext(C)         5304 non-null   float64
 8    Temp_int(C)         5304 non-null   float64
 9    Temp_sea(C)         5304 non-null   float64
 10   EC_sea              5304 non-null   float64
 11   Temp_sea(C) .1      5304 non-null   float64
 12   EC_sea .1           5304 non-null   float64
 13   Temp_sea(C) .2      5304 non-null   float64
 14   EC_sea .2           5304 non-null   float64
dtypes: float64(11), int64(2), object(2)
memory 

### Filtrage des températures

Filtrage de Température de la mer

In [27]:
data.reset_index(inplace=True, drop=True)


data = data[data[' Temp_sea(C) '].between(-20, 100)]
data = data[data[' Temp_sea(C) .1'].between(-20, 100)]
data = data[data[' Temp_sea(C) .2'].between(-20, 100)]
data = data[data[' Temp_sea(C) .2'].between(-20, 100)]


    

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5301 entries, 1 to 5303
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Lat                  5301 non-null   float64
 1    Lng                 5301 non-null   float64
 2    Date                5301 non-null   object 
 3    Time                5301 non-null   object 
 4    Bat %               5301 non-null   int64  
 5    Bat mV              5301 non-null   int64  
 6    Pression_ext(hpa)   5301 non-null   float64
 7    Temp_ext(C)         5301 non-null   float64
 8    Temp_int(C)         5301 non-null   float64
 9    Temp_sea(C)         5301 non-null   float64
 10   EC_sea              5301 non-null   float64
 11   Temp_sea(C) .1      5301 non-null   float64
 12   EC_sea .1           5301 non-null   float64
 13   Temp_sea(C) .2      5301 non-null   float64
 14   EC_sea .2           5301 non-null   float64
dtypes: float64(11), int64(2), object(2)
memory 

Filtrage des températures internes et externes

In [28]:
data.reset_index(inplace=True, drop=True)


data = data[data[' Temp_int(C) '].between(-150, 150)]
data = data[data[' Temp_ext(C) '].between(-150, 150)]
data = data[data[' Pression_ext(hpa) '].between(70, 2000)]

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5301 entries, 0 to 5300
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Lat                  5301 non-null   float64
 1    Lng                 5301 non-null   float64
 2    Date                5301 non-null   object 
 3    Time                5301 non-null   object 
 4    Bat %               5301 non-null   int64  
 5    Bat mV              5301 non-null   int64  
 6    Pression_ext(hpa)   5301 non-null   float64
 7    Temp_ext(C)         5301 non-null   float64
 8    Temp_int(C)         5301 non-null   float64
 9    Temp_sea(C)         5301 non-null   float64
 10   EC_sea              5301 non-null   float64
 11   Temp_sea(C) .1      5301 non-null   float64
 12   EC_sea .1           5301 non-null   float64
 13   Temp_sea(C) .2      5301 non-null   float64
 14   EC_sea .2           5301 non-null   float64
dtypes: float64(11), int64(2), object(2)
me

### Coordonnées : Filtrage des coordonnées
Pour les coordonnées qui sont nulles, il s'agit de moments où le GPS ne capte plus de satellite, on ne souhaite pas éliminer les données mais extrapoler la position à partir des données où les GPS catpe les satellites.


Pour la longitude

In [29]:
data.reset_index(inplace=True, drop=True)

lng_nulles = []
for j in range(len(data)):
    if data[' Lng '][j] == 0:
        lng_nulles.append(j)

print(lng_nulles)

# on traite les cas particuliers de la première et de la dernière donnée en partant du principe que la 2e donnée et l'avant dernière ne sont pas nulles
if data[' Lng '][0] == 0:
    data[' Lng '][0] = data[' Lng '][1]
if data[' Lng '][len(data)-1] == 0:
    data[' Lng '][len(data)-1] = data[' Lng '][len(data)-2]

# cas général
for i in range(1, len(data)-1,1):
    if data[' Lng '][i] == 0:
        if data[' Lng '][i+1]==0:
            data[' Lng '][i] = data[' Lng '][i-1]+ (data[' Lng '][i-1]-data[' Lng '][i-2])  ## traitement du cas ou il y a un enchainement de valeurs nulles
        else:
            data[' Lng '][i] = (data[' Lng '][i-1]+data[' Lng '][i+1])/2


[767, 1334, 1511, 1655, 1686, 1687, 1688, 1689, 1690, 1691, 1766, 2022, 2027, 2029, 2030, 2031, 2032, 2033, 2040, 2051, 2066, 2067, 2068, 2073, 2106, 2324, 2341, 2736, 3066, 3085, 3099, 3109, 3122, 3135, 3136, 3152, 3170, 3172, 3212, 3272, 3273, 3274, 3278, 3294, 3310, 3325, 3349, 3397]


Pour la latitude

In [30]:
data.reset_index(inplace=True, drop=True)

lat_nulles = []
for j in range(len(data)):
    if data['Lat '][j] == 0:
        lat_nulles.append(j)

print(lat_nulles)

# on traite les cas particuliers de la première et de la dernière donnée
if data['Lat '][0] == 0:
    data['Lat '][0] = data['Lat '][1]
if data['Lat '][len(data)-1] == 0:
    data['Lat '][len(data)-1] = data['Lat '][len(data)-2]

# cas général
for i in range(1, len(data)-1,1):
    if data['Lat '][i] == 0:
        if data['Lat '][i+1]==0:
            data['Lat '][i] = data['Lat '][i-1]+ (data['Lat '][i-1]-data['Lat '][i-2])  ## traitement du cas ou il y a un enchainement de valeurs nulles
        else:
            data['Lat '][i] = (data['Lat '][i-1]+data['Lat '][i+1])/2

[767, 1334, 1511, 1655, 1686, 1687, 1688, 1689, 1690, 1691, 1766, 2022, 2027, 2029, 2030, 2031, 2032, 2033, 2040, 2051, 2066, 2067, 2068, 2073, 2106, 2324, 2341, 2736, 3066, 3085, 3099, 3109, 3122, 3135, 3136, 3152, 3170, 3172, 3212, 3272, 3273, 3274, 3278, 3294, 3310, 3325, 3349, 3397]


## Calcul des moyennes et de la salinité

L'instrument SensOcean enregistre plusieurs fois les valeurs de temperature et conductivité de la mer, on doit en faire les moyennes et ensuite calculer la salinité. 

In [31]:
# calcul des moyennes des températures et de EC
temp_mean=(data[' Temp_sea(C) ']+data[' Temp_sea(C) .1']+data[' Temp_sea(C) .2'])/3
ec_mean=(data[' EC_sea ']+data[' EC_sea .1']+data[' EC_sea .2'])/3

# calcul de la salinité
A = [0.0080, -0.1692, 25.3851, 14.0941, -7.0261, 2.7081]
B = [0.0005, -0.0056, -0.0066, -0.0375, 0.0636, -0.0144]
C = [0.6766097, 0.0200564, 0.000110426, -6.9698E-07, 1.0031E-09]
K = 0.0162

def salinity_calculator(temperature, conductivity):
    R_p = 1. # pour les mesures à faible profondeur
    r_t = sum(
        [C[i] * temperature**i for i in range(5)]
    )
    R_t = conductivity / (42.914 * r_t)

    return (
        sum([A[i] * R_t**(int(i)/2.) for i in range(6)])
        + (
            ((temperature - 15)/(1 + K * (temperature - 15)))
            * (sum([B[i] * R_t**(int(i)/2.) for i in range(6)]))
        )
    )

sal_mean=salinity_calculator(temp_mean, ec_mean/1000)

Création de la colonne so_datetime, utile pour la visualisation des données

In [32]:
# merge time and date 
so_datetime=pd.to_datetime(data[' Date ']+data[' Time '],format=' %y/%m/%d  %H:%M:%S ')
print(so_datetime)

0      2021-08-03 09:25:49
1      2021-08-03 09:35:59
2      2021-08-03 09:46:08
3      2021-08-03 09:56:17
4      2021-08-03 10:06:28
               ...        
5296   2021-09-17 11:20:45
5297   2021-09-17 11:31:00
5298   2021-09-17 11:41:35
5299   2021-09-17 11:51:46
5300   2021-09-17 12:01:57
Length: 5301, dtype: datetime64[ns]


## Création d'un nouveau fichier CSV traité et trié

Création des colonnes et ajout de ces dernières dans le csv existant

In [33]:
#ajout des colonnes calculer dans le csv
Temp_mean=temp_mean
Ec_mean=ec_mean
Salinite=sal_mean

data['so_datetime']=so_datetime
data['Temp_mean']=Temp_mean
data['Ec_mean']=Ec_mean
data['Salinite']=Salinite

In [34]:
#voir le  nouveau fichier
data.head()

,Lat,Lng,Date,Time,Bat %,Bat mV,Pression_ext(hpa),Temp_ext(C),Temp_int(C),Temp_sea(C),EC_sea,Temp_sea(C) .1,EC_sea .1,Temp_sea(C) .2,EC_sea .2,so_datetime,Temp_mean,Ec_mean,Salinite
0,64.151752,-21.932455,21/08/03,09:25:49,100,4095,1005.621,12.60,12.438,13.528,39850.0,13.541,39850.0,13.547,39880.0,2021-08-03 09:25:49,13.538667,39860.000000,33.473053
1,64.151746,-21.932064,21/08/03,09:35:59,100,4097,1005.574,12.72,12.500,13.553,39900.0,13.558,39920.0,13.561,39950.0,2021-08-03 09:35:59,13.557333,39923.333333,33.515999
2,64.151808,-21.932357,21/08/03,09:46:08,100,4104,1005.609,12.73,12.562,13.573,39960.0,13.581,39980.0,13.589,40020.0,2021-08-03 09:46:08,13.581000,39986.666667,33.554522
3,64.152692,-21.930707,21/08/03,09:56:17,100,4100,1005.599,-136.72,12.688,13.588,39890.0,13.598,39860.0,13.582,39920.0,2021-08-03 09:56:17,13.589333,39890.000000,33.456743
4,64.159010,-21.919484,21/08/03,10:06:28,100,4103,1005.542,12.84,12.813,13.582,39870.0,13.572,39820.0,13.571,39850.0,2021-08-03 10:06:28,13.575000,39846.666667,33.428743


Création d'un nouveau fichier

In [35]:
# enregistrement dans fichier csv 
data.to_csv('data_traitees.csv', index=False,  sep = ';')